In [1]:
import re
import os
import logging
import sys
import json
import base64
import requests
from bs4 import BeautifulSoup
import pymysql
import aiohttp
import asyncio
from collections import defaultdict
import pyrootutils
import pandas as pd

pyrootutils.setup_root(os.curdir, indicator=".project-root", pythonpath=True)
from aws import connect, rds_insert, s3_upload
from extras import constants, paths
from lmood.utils import search, category, utils

In [2]:
json_path = '/Users/sinjeong-yeol/Documents/aws/crawling-db.json'
with open(json_path) as f:
    json_object = json.load(f)

In [3]:
page_url = constants.LMOOD_PAGE_URL + "&page=1"

In [4]:
product_li = search.get_product_li(page_url)
product_url, thumbnail_image_url = search.get_url(product_li[0])
product_url, thumbnail_image_url

('https://lmood.co.kr/product/detail.html?product_no=2867&cate_no=198&display_group=1',
 'https://lmood.co.kr/web/product/medium/202304/de909ee6d9895c08004617cfa4487a90.jpg')

In [5]:
response = requests.get(product_url, headers={"User-Agent": "Mozilla/5.0"})
html = response.text
soup = BeautifulSoup(html, "html.parser")
name = soup.select('h3.product-name')[0].get_text().strip()

In [6]:
soup.select('div#span_prd_price_sale_text')[0].get_text().strip()

'₩88,200'

In [7]:
category.classify(name)

7

In [8]:
data = soup.find('p', {"class": "ko"}).find_all(string=True)
data = '\n'.join(list(map(lambda x: x.strip(), data)))

In [9]:
table = soup.select('div.guideBoard > table')[0]

In [11]:
search.crawling_image(soup)

['https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/1-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/2-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/3-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/4-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/5-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/1-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/2-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/3-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/4-1.jpg',
 'https://lmood.co.kr/renewal/23/HS/02_DROP/DIVE%20HALF%20SHIRTS/CLEAR%20BLACK/5-1.jpg']

## test

In [1]:
import os
import pyrootutils

pyrootutils.setup_root(os.curdir, indicator=".project-root", pythonpath=True)
from lmood.utils import utils, search, category
from extras import constants, paths
from aws import connect, rds_insert, s3_upload

In [2]:
s3 = connect.connect_s3()
conn, cursor = connect.connect_RDS()

In [3]:
product_li = search.get_product_li(constants.LMOOD_PAGE_URL + "&page=1")[0]

In [4]:
product_url, thumbnail_image_url = search.get_url(product_li)

In [5]:
product_id, img_url_lst = search.crawling_product(product_url, s3, conn, cursor)

In [6]:
search.s3_rds_image(product_id, thumbnail_image_url, img_url_lst, s3, conn, cursor)

In [7]:
connect.close_RDS(conn, cursor)